In [57]:
def week3_ans_func(problem_set):
    
    qc=solution(problem_set)
    
    return qc

In [58]:
# Submission code
from qc_grader import grade_ex3, prepare_ex3, submit_ex3

# Execute your circuit with following prepare_ex3() function.
# The prepare_ex3() function works like the execute() function with only QuantumCircuit as an argument.
job = prepare_ex3(week3_ans_func)

result = job.result()
counts = result.get_counts()
original_problem_set_counts = counts[0]

original_problem_set_counts
# The bit string with the highest number of observations is treated as the solution.

Running week3_ans_func...
Computing cost...
Starting experiments. Please wait...
You may monitor the job (id: 5fc44b10d3dc27001a674fa5) status and proceed to grading when it successfully completes.


{'0000': 34,
 '0001': 38,
 '0010': 40,
 '0011': 33,
 '0100': 32,
 '0101': 473,
 '0110': 44,
 '0111': 40,
 '1000': 30,
 '1001': 31,
 '1010': 34,
 '1011': 31,
 '1100': 37,
 '1101': 33,
 '1110': 34,
 '1111': 36}

In [59]:
# Check your answer by executing following code.
# The quantum cost of the QuantumCircuit is obtained as the score. The lower the cost, the better.
grade_ex3(job)

Grading your answer. Please wait...

Congratulations 🎉! Your answer is correct.
Your score is 98993.
The lower your score the better!
Feel free to submit your answer.


In [45]:
# Submit your results by executing following code. You can submit as many times as you like during the period. 
submit_ex3(job)

Submitting your answer. Please wait...

Success 🎉! Your answer has been submitted.
Congratulations! You have rescued Dr. Ryoko from the quantum realm. The bright "quantum future" is ahead.


In [56]:
from qiskit import *
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import IBMQ, Aer, execute
from qiskit.visualization import plot_histogram
import numpy as np
from itertools import permutations

problem_set = \
    [[['0', '2'], ['1', '0'], ['1', '2'], ['1', '3'], ['2', '0'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '2'], ['2', '2'], ['3', '0'], ['3', '3']],
    [['0', '0'], ['1', '1'], ['1', '3'], ['2', '0'], ['3', '2'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '1'], ['1', '3'], ['3', '2'], ['3', '3']],
    [['0', '2'], ['1', '0'], ['1', '3'], ['2', '0'], ['3', '2'], ['3', '3']],
    [['1', '1'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '1'], ['3', '3']],
    [['0', '2'], ['0', '3'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '3']],
    [['0', '0'], ['0', '3'], ['1', '2'], ['2', '2'], ['2', '3'], ['3', '0']],
    [['0', '3'], ['1', '1'], ['1', '2'], ['2', '0'], ['2', '1'], ['3', '3']],
    [['0', '0'], ['0', '1'], ['1', '3'], ['2', '1'], ['2', '3'], ['3', '0']],
    [['0', '1'], ['0', '3'], ['1', '2'], ['1', '3'], ['2', '0'], ['3', '2']],
    [['0', '0'], ['1', '3'], ['2', '0'], ['2', '1'], ['2', '3'], ['3', '1']],
    [['0', '1'], ['0', '2'], ['1', '0'], ['1', '2'], ['2', '2'], ['2', '3']],
    [['0', '3'], ['1', '0'], ['1', '3'], ['2', '1'], ['2', '2'], ['3', '0']],
    [['0', '2'], ['0', '3'], ['1', '2'], ['2', '3'], ['3', '0'], ['3', '1']],
    [['0', '1'], ['1', '0'], ['1', '2'], ['2', '2'], ['3', '0'], ['3', '1']]]

def write_boards(qc, b, a, boards):
    boards = [[[str(j),str(i)] in board for j in range(4) for i in range(4)] for board in boards]
    for i, board in enumerate(boards):
        address_pattern = list(map(bool, map(int, list(format(i, '04b')))))
        
        # add NOT if needed for address
        for x, maybe in enumerate(address_pattern): 
            if not maybe:
                qc.x(b[x])
                
        for j, asteroid in enumerate(board):
            if asteroid:
                qc.mct(b, a[j], mode='noancilla')
            
        # undo all NOT if needed for address
        for x, maybe in enumerate(address_pattern): 
            if not maybe:
                qc.x(b[x])
                
        qc.barrier()

def oracle(qc, a, o,z):
    for i in range(4):
        i1 = [0,1,2,3]
        i1.remove(i)
        for j in i1:
            i2 = list(i1)
            i2.remove(j)
            for k in i2:
                i3 = list(i2)
                i3.remove(k)
                for l in i3:
                    qc.mct([a[i], a[j+4], a[k+8], a[l+12]], o,z, mode='v-chain-dirty')
                    #mct(control_qubits, target_qubit, ancilla_qubits=None, mode='noancilla')

                    
def diffusion(qc, qr):
    last = len(qr) - 1
    
    qc.h(qr)
    qc.x(qr)
    qc.h(qr[last])
    qc.mct(qr[0:last], qr[last])
    qc.h(qr[last])
    qc.x(qr)
    qc.h(qr)
    
def solution(problem_set):
    b = QuantumRegister(4, name='boards')
    a = QuantumRegister(16, name='asteroids')
    o = QuantumRegister(1, name='oracle')
    z = QuantumRegister(2, name='ancilla')
    
    c = ClassicalRegister(4, name='output')
    
    qc = QuantumCircuit(b,a,o,c,z)
    
    qc.x(o)
    qc.h(o)
    
    qc.h(b)
    
    write_boards(qc, b, a, problem_set)
    
    oracle(qc, a, o,z)
    
    write_boards(qc, b, a, problem_set)
    
    diffusion(qc, b)
    
    qc.h(o)
    qc.x(o)
    
    qc.measure(b,c)
    return qc